In [98]:
import pandas as pd

In [99]:
pedidos_comida=pd.read_csv("food_orders_new_delhi.csv")

In [100]:
print(pedidos_comida.head())


   Order ID Customer ID Restaurant ID  Order Date and Time  \
0         1       C8270         R2924  2024-02-01 01:11:52   
1         2       C1860         R2054  2024-02-02 22:11:04   
2         3       C6390         R2870  2024-01-31 05:54:35   
3         4       C6191         R2642  2024-01-16 22:52:49   
4         5       C6734         R2799  2024-01-29 01:19:30   

  Delivery Date and Time  Order Value  Delivery Fee    Payment Method  \
0    2024-02-01 02:39:52         1914             0       Credit Card   
1    2024-02-02 22:46:04          986            40    Digital Wallet   
2    2024-01-31 06:52:35          937            30  Cash on Delivery   
3    2024-01-16 23:38:49         1463            50  Cash on Delivery   
4    2024-01-29 02:48:30         1992            30  Cash on Delivery   

  Discounts and Offers  Commission Fee  Payment Processing Fee  \
0            5% on App             150                      47   
1                  10%             198                  

In [101]:
print(pedidos_comida.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Order ID                1000 non-null   int64 
 1   Customer ID             1000 non-null   object
 2   Restaurant ID           1000 non-null   object
 3   Order Date and Time     1000 non-null   object
 4   Delivery Date and Time  1000 non-null   object
 5   Order Value             1000 non-null   int64 
 6   Delivery Fee            1000 non-null   int64 
 7   Payment Method          1000 non-null   object
 8   Discounts and Offers    815 non-null    object
 9   Commission Fee          1000 non-null   int64 
 10  Payment Processing Fee  1000 non-null   int64 
 11  Refunds/Chargebacks     1000 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 93.9+ KB
None


#Data cleaning 
conversion columnas hora y fecha a datetime

In [102]:
pedidos_comida['order date and time']=pd.to_datetime(pedidos_comida['Order Date and Time'])
pedidos_comida['Delivery Date and Time']=pd.to_datetime(pedidos_comida['Delivery Date and Time'])

#verificar valores monetarios en formato correcto para realizar calculos
se  crea una funcion para extraer valores numericos desde 'discount and offers'

In [103]:
def extraer_descuento(discount_str):
    if 'off' in discount_str:
        #monto fijo
        return float(discount_str.split(' ')[0])
    elif '%'in discount_str:
        #porcentaje descuento
        return float(discount_str.split('%')[0])
    else:
        #sin descuento
        return 0.0

#aplicar la funcion para crear  columna 'nuevo descuento'

In [104]:
pedidos_comida['discount percentage'] = pedidos_comida['Discounts and Offers'].astype(str).apply(lambda x: extraer_descuento(x))


#para porcentajes de descuento, calcular el descuento basado en el precio del pedido

In [105]:
pedidos_comida['discount amount']=pedidos_comida.apply(lambda x: (x['Order Value'] * x['discount percentage'] /100)
                                                                  if x['discount percentage'] >1
                                                                  else x['discount percentage'], axis=1)

#ajustar 'discount amount' para descuentos fijos drectamente especificados en la columna 'discount offers'

In [106]:
pedidos_comida['discount amount']=pedidos_comida.apply(lambda x: x['discount amount'] if x['discount percentage'] <=1
                                                       else x['Order Value'] *x['discount percentage'] /100, axis=1)

In [107]:
print(pedidos_comida[['Order Value','Discounts and Offers', 'discount percentage','discount amount']].head(), pedidos_comida.dtypes)

   Order Value Discounts and Offers  discount percentage  discount amount
0         1914            5% on App                  5.0            95.70
1          986                  10%                 10.0            98.60
2          937         15% New User                 15.0           140.55
3         1463                  NaN                  0.0             0.00
4         1992         50 off Promo                 50.0           996.00 Order ID                           int64
Customer ID                       object
Restaurant ID                     object
Order Date and Time               object
Delivery Date and Time    datetime64[ns]
Order Value                        int64
Delivery Fee                       int64
Payment Method                    object
Discounts and Offers              object
Commission Fee                     int64
Payment Processing Fee             int64
Refunds/Chargebacks                int64
order date and time       datetime64[ns]
discount percentage    

#data cleaning listo
order date and time y delivery date and time se trasnformaron  a formato datetime
una nueva columna , discount amount , fue creada en base a discount and offers , esto se hizo extrayendo porcentahe de descuentos o montos de descuento fijos, y aplicandolod al valor del pedido.
se agrego decuento de porcentaje para representar el descuento a aplicar o el monto fijo durectamente. 

#"Análisis de Costos y Rentabilidad".